In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pancancer-8-dataset/genes.csv
/kaggle/input/pancancer-8-dataset/8_cancers_merged.csv
/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv


In [2]:
import pandas as pd

# Load your data
data = pd.read_csv("/kaggle/input/pancancer-8-dataset/Preprocessed_8_cancer_genes.csv")

# Split the data into features and target labels
X = data.drop(columns=["cancer"])  # All gene expression features
y = data["cancer"]  # Target labels (cancer types)


In [3]:
X = X.iloc[:,1:]
X

,DDX11L1,WASH7P,MIR6859-1,MIR1302-2HG,MIR1302-2,FAM138A,OR4G4P,OR4G11P,OR4F5,AL627309.1,...,SNORA70.10,PPP1R12BP1,CCNQP2,CTBP2P1,AMD1P2.1,MT-TW,MT-ND3,MT-TS2,MT-TE,MT-TP
0,0.290465,2.346836,0.410837,1.652246,0.291915,0.101839,0.207631,-0.205970,0.122728,0.896219,...,0.396830,-0.217605,-0.604337,-0.307557,-0.121871,-0.341261,-0.591108,0.387197,0.131472,0.475067
1,-1.091393,-0.178510,0.713063,1.192765,-0.194697,-0.782244,0.180330,1.152590,-0.157871,0.749253,...,-0.032151,-0.544846,-0.604337,-0.427793,0.458588,-0.190891,0.520403,-0.657352,1.812840,0.717846
2,0.892004,-0.579981,0.337297,2.489210,1.686052,-0.921599,-0.837422,-0.933123,-0.684331,1.029887,...,-0.453687,-0.400647,-0.436683,-0.479891,-0.752229,-0.230458,-0.269168,-0.005220,1.715524,-0.570044
3,-0.328462,1.438270,-0.146134,1.618618,1.825156,-0.219842,0.081615,-0.214707,-0.505851,0.711919,...,-0.093735,-0.032827,-0.393916,-0.453396,-0.740037,-0.061655,-0.579535,-0.458516,1.698385,-0.706931
4,-0.704519,-0.202510,-0.499540,0.545313,-0.005537,-1.354323,-0.457143,-0.780083,-0.823027,1.021426,...,-0.870362,-0.655694,0.606534,1.198791,-0.607317,0.135548,0.980353,0.062700,2.348368,-1.003697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4106,0.469028,-0.470937,0.452521,-0.929403,0.479663,-0.254867,0.649311,1.597366,-0.598738,1.370125,...,-0.062616,-0.867095,-0.604337,-0.471802,0.304774,-0.179607,2.261286,-0.213173,0.757743,1.666541
4107,-0.578430,-0.579981,-0.480992,-1.047113,0.095164,1.138017,-0.340487,1.045525,-0.903839,-0.304706,...,-0.951742,-0.345173,-0.238886,-0.419961,-0.499946,-0.341261,-0.262022,-1.020234,-1.716828,0.024736
4108,-1.897611,-0.579981,-3.170477,-3.265805,-2.400640,-0.100255,-1.870222,-2.658061,-2.670656,-2.895116,...,-1.105005,-0.531544,-0.604337,-0.366625,-1.808325,-0.341261,-0.796970,-1.300475,-2.741872,-0.759130
4109,0.429964,0.303652,0.429072,1.183050,1.616065,-1.406358,0.241902,0.230997,0.048474,1.342270,...,1.183762,-0.867095,-0.604337,-0.468641,0.771159,-0.158472,-0.796970,2.126268,1.019680,1.210609


In [4]:
X = X.dropna(how="all")  # Removes rows where *all* values are NaN
print(np.isnan(X).sum().sum())  # Should be 0


0


In [5]:
X.shape

(4104, 30815)

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold

# Assuming X is a DataFrame with normalized data
# Gene names are the columns of X
gene_names = X.columns  # Get original feature names (gene names)

# Define input shape
input_dim = X.shape[1]  
feature_percentages = [0.0025, 0.005, 0.01]  # 1%, 1.5%, 2% of total features

# Number of folds for cross-validation
n_splits = 5  # You can adjust this depending on your dataset size

# Dictionary to store results
selected_features_dict = {}

# Create KFold cross-validation generator
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

for feature_percent in feature_percentages:
    top_features_count = int(input_dim * feature_percent)  # Calculate the number of features to select
    print(f"\nTraining Autoencoder with Top {feature_percent * 100}% ({top_features_count} features)")

    # Initialize an empty list to store selected features from all folds
    all_selected_features = []

    # Cross-validation loop
    for fold, (train_index, val_index) in enumerate(kf.split(X)):
        print(f"\nTraining fold {fold+1}/{n_splits}")

        # Split data into training and validation sets for this fold
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]

        # Encoder
        input_layer = Input(shape=(input_dim,))
        x = Dense(1024, activation=None)(input_layer)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.3)(x)

        x = Dense(512, activation=None)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.3)(x)

        x = Dense(256, activation=None)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        latent_space = Dense(top_features_count, activation='linear', name='latent_space')(x)

        # Decoder
        x = Dense(256, activation=None)(latent_space)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        x = Dense(512, activation=None)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        x = Dense(1024, activation=None)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.3)(x)

        decoded = Dense(input_dim, activation='linear')(x)

        # Build and compile the autoencoder
        autoencoder = Model(inputs=input_layer, outputs=decoded)
        autoencoder.compile(optimizer=Adam(learning_rate=0.001, clipvalue=1.0), loss='mse')

        # Callbacks: Early stopping & Reduce LR on plateau
        early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

        # Train the model with verbose set to 1 (progress bar)
        history = autoencoder.fit(
            X_train, X_train,  # Use already normalized X_train
            epochs=100, batch_size=128, 
            validation_data=(X_val, X_val),
            callbacks=[early_stop, reduce_lr], 
            verbose=1  # Show progress bar for training
        )

        # Extract latent features (using the encoder model)
        encoder = Model(inputs=input_layer, outputs=latent_space)
        X_latent = encoder.predict(X_val)

        # Calculate variance of latent space representations
        feature_variances = np.var(X_latent, axis=0)  # Variance across samples for each latent feature
        top_feature_indices = np.argsort(feature_variances)[-top_features_count:]  # Get indices of top features

        # Select the top features
        selected_features = X_latent[:, top_feature_indices]
        
        # Append the selected features for this fold
        all_selected_features.append(selected_features)

    # Ensure that all arrays in `all_selected_features` have the same shape
    max_samples = max([arr.shape[0] for arr in all_selected_features])  # Get max number of samples across folds
    
    # Pad all arrays to have the same number of samples (max_samples)
    padded_features = []
    for arr in all_selected_features:
        if arr.shape[0] < max_samples:
            # Pad the array to match the shape
            padded_arr = np.pad(arr, ((0, max_samples - arr.shape[0]), (0, 0)), mode='constant', constant_values=np.nan)
            padded_features.append(padded_arr)
        else:
            padded_features.append(arr)

    # Convert the padded list into a NumPy array
    all_selected_features_array = np.array(padded_features)

    # Average the selected features across all folds (axis=0 corresponds to folds)
    all_selected_features_mean = np.mean(all_selected_features_array, axis=0)

    # Map the selected features to gene names
    selected_feature_names = gene_names[top_feature_indices]  # Get corresponding gene names for the selected features

    # Create DataFrame with selected features and their gene names
    selected_features_df = pd.DataFrame(all_selected_features_mean, columns=selected_feature_names)

    # Save as CSV
    filename = f"top_{str(float(feature_percent*100))}_percent_features_cv.csv"
    selected_features_df.to_csv(filename, index=False)

    # Store in dictionary
    selected_features_dict[feature_percent] = selected_features_df

print("\nFeature selection with cross-validation completed!")



Training Autoencoder with Top 0.25% (77 features)

Training fold 1/5
Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 298ms/step - loss: 0.7625 - val_loss: 12.5170 - learning_rate: 0.0010
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.5229 - val_loss: 5.2011 - learning_rate: 0.0010
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - loss: 0.5057 - val_loss: 1.9007 - learning_rate: 0.0010
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 0.4777 - val_loss: 0.7825 - learning_rate: 0.0010
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.4630 - val_loss: 1.0330 - learning_rate: 0.0010
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.4627 - val_loss: 0.5261 - learning_rate: 0.0010
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.4392 - val_loss: 0.5060 - learning_rate: 0.0010
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.4408 - val_loss: 0.4424 - learning_rate: 0.0010
Epoch 9/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/